In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer

# import seaborn as sns
import time
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
def getTextFromFiles(df, data_path, depression, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df = df.append({'text': file1, 'depression': int(depression)}, ignore_index=True)

    return df

# test dataset 읽기
def getTextFromFiles_Test(df_test, data_path, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df_test = df_test.append({'text': file1}, ignore_index=True)

    return df_test

In [59]:
def dataPreprocessingForX(df, columnName1):
  #   읽어들인 데이터를 모두 소문자화 시킨다
  df[columnName1] = df[columnName1].map(lambda text: text.lower())
  # 단어 단위로 나눈다(word tokenzing)
  df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
  # stoptwords를 이용하여 불필요한 단어 제거
  stop_words = set(nltk.corpus.stopwords.words('english'))
  df[columnName1] = df[columnName1].map(lambda tokens: [w for w in tokens if not w in stop_words])
  # regular expression을 이용한 특수문자 제거
  df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))
  df[columnName1] = df[columnName1].map(lambda text: re.sub('[^A-Za-z]+', ' ', text))
  # word stemmer를 톨해 단어를 원문화 시킨다.
  df[columnName1] = df[columnName1].map(lambda text: nltk.tokenize.word_tokenize(text))
  ps = PorterStemmer()
  lemmatizer = WordNetLemmatizer()
  # print(df[columnName1][0])
  df[columnName1] = df[columnName1].map(lambda text: [lemmatizer.lemmatize(i) for i in text])
  df[columnName1] = df[columnName1].map(lambda text: ' '.join(text))

# 파일이 제대로 읽혔는지 확인하는 function
def checkfilesCounts(data_path):
    print(data_path+" have: ", len(os.listdir(data_path)),"files")

# 개인 pc에서 실행시
# data_path_d = "reddit_depression"
# data_path_nd = "reddit_non_depression"
# data_path_d_test = "reddit_depression_testset"
# collab에서 실행시
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"

checkfilesCounts(data_path_d)
checkfilesCounts(data_path_nd)
#
# # 데이터 전처리
df = pd.DataFrame(columns=['text', 'depression'])
df = getTextFromFiles(df, data_path_d, 1, 500)


/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression have:  1293 files
/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression have:  548 files


In [60]:
# # 이시점까진 우울증 글만 추가
df = getTextFromFiles(df, data_path_nd, 0, 500)
dataPreprocessingForX(df, 'text')

df['depression'] = df['depression'].astype('int32')

In [ ]:

sns.countplot(df['depression'])
plt.xlabel('Label')
plt.title('Number of Non depression and depression messages')

In [75]:
X = df['text'].to_numpy()
Y = df['depression'].to_numpy()
# le = LabelEncoder()
# Y = le.fit_transform(Y)
# Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

max_words = 100
max_len = 150

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [76]:
print(X_train.shape)
print(sequences)
print(sequences_matrix.shape)

(850,)
[[47, 86, 7, 4, 90, 54, 3, 7, 4, 79, 28, 20, 7, 13, 90, 49, 37, 21, 7, 4, 69, 22, 41, 8, 19, 70, 6, 54, 3, 14, 5, 12, 84, 3, 22, 28, 42, 69], [4, 17, 66, 2, 1, 4, 66, 2, 1, 43, 15, 90, 84, 2, 1, 10, 22, 2, 1, 10, 23, 15, 84, 13, 15, 12, 66, 2, 1, 17, 2, 1, 10, 97, 8, 5, 13, 34, 2, 1, 11, 96, 12, 84, 2, 1, 83, 99, 83, 55, 2, 1, 19, 65, 48, 13, 2, 1, 25, 12, 84, 11], [46, 62, 7, 4, 28, 5, 62, 46, 22, 22, 28, 7, 4, 5, 28, 85, 29, 45, 46, 51, 28, 30, 28, 24, 67, 65, 7, 4, 81, 39, 7, 4, 39, 18, 1, 1, 54, 3, 5, 22, 28, 55, 24, 24, 22, 1, 21, 47, 20, 24, 19, 22, 24, 33, 1, 5, 28, 24, 22, 24], [47, 23, 33, 1, 10, 47, 68, 69, 35, 7, 4, 15, 7, 4, 47, 77, 15], [61, 6, 70, 10, 54, 3, 7, 4, 14, 7, 4, 52, 7, 4, 1, 17, 30, 54, 3, 33, 1, 10, 22, 85, 27, 6, 18, 30, 16, 8, 5, 7, 4, 7, 60, 67, 27, 22, 58, 52, 55, 62, 41, 9, 86, 49, 10, 62, 52, 6, 41, 40, 16], [6, 5, 14, 36, 88, 18, 6, 92, 15, 92, 8, 46, 92, 6, 4, 5, 14, 3, 6, 6, 53, 92, 19, 5, 2, 1, 19, 10, 77, 6, 34, 52], [12, 69, 7, 4, 58, 54, 3

In [77]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [78]:
model = tf.keras.models.Sequential()
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 50)           5000      
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                

In [79]:
model.fit(sequences_matrix,Y_train,batch_size=4,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
170/170 [==============================] - 7s 43ms/step - loss: 0.6124 - accuracy: 0.6529 - val_loss: 0.7523 - val_accuracy: 0.6882
Epoch 2/10
170/170 [==============================] - 7s 42ms/step - loss: 0.5022 - accuracy: 0.7985 - val_loss: 0.3926 - val_accuracy: 0.8353
Epoch 3/10
170/170 [==============================] - 7s 43ms/step - loss: 0.4068 - accuracy: 0.8206 - val_loss: 0.3662 - val_accuracy: 0.8588
Epoch 4/10
170/170 [==============================] - 7s 43ms/step - loss: 0.4026 - accuracy: 0.8368 - val_loss: 0.3307 - val_accuracy: 0.8647
Epoch 5/10
170/170 [==============================] - 7s 43ms/step - loss: 0.3676 - accuracy: 0.8529 - val_loss: 0.3193 - val_accuracy: 0.8706
Epoch 6/10
170/170 [==============================] - 7s 43ms/step - loss: 0.3359 - accuracy: 0.8882 - val_loss: 0.3181 - val_accuracy: 0.8706
Epoch 7/10
170/170 [==============================] - 7s 43ms/step - loss: 0.3273 - accuracy: 0.8721 - val_loss: 0.3414 - val_accuracy: 0.8706

In [81]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [82]:
accr = model.evaluate(test_sequences_matrix,Y_test)

5/5 [==============================] - 0s 15ms/step - loss: 0.3524 - accuracy: 0.8467


In [83]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.352
  Accuracy: 0.847


In [69]:
from keras.preprocessing.text import Tokenizer

text = 'Yours and mine ancestors had run after a mastodons or wild boar, like an olympic sprinter, with a spear in hand covering themselves with leaves and tiger skin, for their breakfast'

tokemizer = Tokenizer(num_words = 30) # numwords is number of top common words to take. Rest words are marked as all 0s for one-hot
tokemizer.fit_on_texts(text)
print(tokemizer.word_index)

print(tokemizer.texts_to_matrix(text[0].split(),mode='binary'))

{'a': 1, 'e': 2, 'r': 3, 's': 4, 'n': 5, 'i': 6, 't': 7, 'o': 8, 'd': 9, 'h': 10, 'l': 11, 'm': 12, 'c': 13, 'f': 14, 'w': 15, 'k': 16, 'p': 17, 'v': 18, 'y': 19, 'u': 20, 'b': 21, 'g': 22}
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]
